In [1]:
JIRA_CREDENTIALS = {
    "token_auth": "",
    "server": "https://issues.apache.org/jira/"
}

GITHUB_CREDENTIALS = {
    "Authorization": "", "User-Agent": "rok",
    "Accept": "application/vnd.github.golden-comet-preview+json"}

In [2]:
import csv, pickle, re, time
from string import punctuation
import jira2markdown
from jira2markdown.markup.links import Mention
from jira2markdown.markup.base import AbstractMarkup
from jira import JIRA
from multiprocessing import Pool, cpu_count
from pyparsing import (
    CaselessLiteral,
    Char,
    Combine,
    FollowedBy,
    Optional,
    ParserElement,
    ParseResults,
    PrecededBy,
    SkipTo,
    StringEnd,
    StringStart,
    Suppress,
    White,
    Word,
    alphanums,
)
import requests


raw_jira_issues_filename = 'raw_jira_issues.pickle'
raw_jira_watchers_filename = 'raw_jira_watchers.pickle'
raw_jira_remote_links_filename = "raw_jira_remote_links.pickle"
raw_github_prs_filename = "raw_github_prs.pickle"
translated_markup_filename = "translated_markdown.pickle"
jira_to_github_user_mapping_file = 'jira-to-github-user-mapping.csv'

RELEASE_ORDER = (
    '0.1.0', '0.2.0', '0.3.0', 'JS-0.3.0', 'JS-0.3.1', '0.4.0', 'JS-0.4.0',
    '0.4.1', 'JS-0.4.1', '0.5.0', '0.6.0', '0.7.0', '0.7.1', '0.8.0',
    '0.9.0', '0.10.0', '0.11.0', '0.11.1', '0.12.0', '0.12.1', '0.13.0',
    '0.14.0', '0.14.1', '0.15.0', '0.15.1', '0.16.0', '0.17.0', '0.17.1',
    '1.0.0', '1.0.1', '2.0.0', '3.0.0', '3.0.1', '4.0.0', '4.0.1', '5.0.0',
    '5.0.1', '6.0.0', '6.0.1', '6.0.2', '6.0.3', '7.0.0', '7.0.1', '7.0.2',
    '8.0.0', '8.0.1', '9.0.0', '9.0.1', '10.0.0', '10.0.1', '10.0.2',
    '11.0.0', '12.0.0'
)

ISSUETYPE_MAP = {
    "Bug": "Type: bug",
    "Improvement": "Type: enhancement",
    "Wish": "Type: enhancement",
    "New Feature": "Type: enhancement",
    "Task": "Type: task",
    "Sub-task": "Type: task",
    "Test": "Type: test"
}

GITHUB_LABELS = (
    "Component: Archery", "Component: Benchmarking", "Component: C",
    "Component: C#", "Component: C++", "Component: C++ - Gandiva",
    "Component: C++ - Plasma", "Component: Continuous Integration",
    "Component: Developer Tools", "Component: Documentation",
    "Component: FlightRPC", "Component: Format", "Component: GLib",
    "Component: Go", "Component: GPU", "Component: Integration",
    "Component: Java", "Component: JavaScript", "Component: Julia",
    "Component: MATLAB", "Component: Other", "Component: Packaging",
    "Component: Parquet", "Component: Python", "Component: R",
    "Component: Release", "Component: Ruby", "Component: Rust",
    "Component: Rust - Ballista", "Component: Rust - DataFusion",
    "Component: Website", "Component: Wiki", "dependencies",
    "good-first-issue", "hacktoberfest-accepted", "java", "javascript",
    "lang-go", "needs-rebase", "ready-for-review", "Type: bug",
    "Type: enhancement", "Type: task", "Type: test", "Type: usage",
    "WIP"
)

milestone_url = "https://api.github.com/repos/apache/arrow/milestones"
raw_milestone_map = requests.get(milestone_url, params={"state": "all"}, headers=GITHUB_CREDENTIALS)
MILESTONE_MAP = {x["title"]: x["number"] for x in raw_milestone_map.json()}

testing_milestone_url = "https://api.github.com/repos/datatart/import_dry_run_3/milestones"
raw_testing_milestone_map = requests.get(testing_milestone_url, params={"state": "all"}, headers=GITHUB_CREDENTIALS)
TESTING_MILESTONE_MAP = {x["title"]: x["number"] for x in raw_testing_milestone_map.json()}

MILESTONE_MAP = TESTING_MILESTONE_MAP

MIGRATION_NOTE = "\n\n**Note**: *This issue was originally created as [{issue_key}]({jira_url}). " \
    "Please see the " \
    "[migration documentation](https://gist.github.com/toddfarmer/12aa88361532d21902818a6044fda4c3) " \
    "for further details.*"

USER_MAPPING = {}
with open(jira_to_github_user_mapping_file, newline="") as f:
    reader = csv.reader(f)
    for row in reader:
        USER_MAPPING[row[0]] = row[2]
        USER_MAPPING[row[1]] = row[2]


OWNER = "datatart"
REPO = "import_dry_run_2"
IMPORT_URL = f"https://api.github.com/repos/{OWNER}/{REPO}/import/issues"

GITHUB_PROJECT_URL = "https://github.com/apache/arrow/pull/"

PROJECT_NAME = "ARROW"

In [3]:
# This is to check for assignable users. We currently don't need it.

# def get_assignable_users(users):
#     user_can_be_assignee = []

#     with requests.Session() as s:
#         for user in users:
#             url = f"https://api.github.com/repos/apache/arrow/assignees/{user}"
#             response = s.get(url, headers=GITHUB_CREDENTIALS)

#             if response.status_code == 204:
#                 user_can_be_assignee.append(user)

#             elif response.status_code == 404:
#                 continue

#             elif response.status_code == 403:
#                 reset_time = int(response.headers["X-RateLimit-Reset"])
#                 wait_time = reset_time - round(time.time() + .5)
#                 if wait_time > 0:
#                     print(f"Throttled on getting {url}, sleeping for {wait_time} seconds.")
#                     print("Response was: ", response.json())
#                     time.sleep(wait_time)

#             else:
#                 response.raise_for_status()

# USER_CAN_BE_ASSIGNEE = get_assignable_users(USER_MAPPING.values())

# Get Jira issue data and cache it to pickle

In [4]:
# %%time
# conn = JIRA(**JIRA_CREDENTIALS)
# issues = conn.search_issues(f"project = {PROJECT_NAME} order by key", maxResults = False, fields = '*all')

# watchers = {}
# for i, issue in enumerate(issues):
#     if i % 1000 == 0:
#         print(f"Getting watchers for {issue.key} [{i}/{len(issues)}].")
#     watchers[issue.id] = conn.watchers(issue.id)

# remote_links = {}
# for i, issue in enumerate(issues):
#     if i % 1000 == 0:
#         print(f"Getting remote links for {issue.key} [{i}/{len(issues)}].")
#     remote_links[issue.id] = conn.remote_links(issue)


# with open(raw_jira_issues_filename, 'wb') as handle:
#     pickle.dump(issues, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(raw_jira_watchers_filename, 'wb') as handle:
#     pickle.dump(watchers, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(raw_jira_remote_links_filename, 'wb') as handle:
#     pickle.dump(remote_links, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(raw_jira_issues_filename, 'rb') as handle:
    issues = pickle.load(handle)

with open(raw_jira_watchers_filename, 'rb') as handle:
    WATCHERS = pickle.load(handle)

with open(raw_jira_remote_links_filename, 'rb') as handle:
    REMOTE_LINKS = pickle.load(handle)

# Markdown correction

In [5]:
class MigratedMention(AbstractMarkup):
    def action(self, tokens: ParseResults) -> str:
        username = self.usernames.get(tokens.accountid)
        return f"`[~{tokens.accountid}]`" if username is None else f"@{username}"

    @property
    def expr(self) -> ParserElement:
        MENTION = Combine(
            "["
            + Optional(
                SkipTo("|", failOn="]") + Suppress("|"),
                default="",
                )
            + "~"
            + Optional(CaselessLiteral("accountid:"))
            + Word(alphanums + ":-").setResultsName("accountid")
            + "]",
            )
        return (
                (StringStart() | Optional(PrecededBy(White(), retreat=1), default=" "))
                + MENTION.setParseAction(self.action)
                + (StringEnd() | Optional(FollowedBy(White() | Char(punctuation, excludeChars="[") | MENTION), default=" "))
        )

LEADING_SPACE_HASH_PATTERN = re.compile(r"\n\s(#+\s+\S.*)")
ELEMENTS = jira2markdown.elements.MarkupElements()
ELEMENTS.replace(Mention, MigratedMention)

def handle_leading_space_before_hash(txt):
    return txt

def translate_markup(issue):
    if issue.fields.description:
        description = issue.fields.description
    else:
        description = ""

    description = re.sub(LEADING_SPACE_HASH_PATTERN, r"\n\1", description)
    text = jira2markdown.convert(description, elements=ELEMENTS, usernames=USER_MAPPING)

    for attachment in issue.fields.attachment:
        text = text.replace(f"![{attachment.filename}]({attachment.filename})",
                            f"![{attachment.filename}]({attachment.content})")

    comments = {}
    for comment in issue.fields.comment.comments:
        # Skip ASF GitHub Bot comments per https://github.com/apache/arrow/issues/14648
        if comment.author.name == "githubbot":
            continue
        comment_body = re.sub(LEADING_SPACE_HASH_PATTERN, r"\n\1", comment.body)
        comment_text = jira2markdown.convert(comment_body, elements=ELEMENTS, usernames=USER_MAPPING)

        for attachment in issue.fields.attachment:
            comment_text = comment_text.replace(f"![{attachment.filename}]({attachment.filename})",
                                                f"![{attachment.filename}]({attachment.content})")
        comments[comment.id] = comment_text

    return (issue.key, {"description": text, "comments": comments})

In [6]:
%%time

# with Pool(processes=int(cpu_count() / 2)) as pool:
#     TRANSLATED_MARKUP = pool.map_async(translate_markup, issues, chunksize=100).get()
# TRANSLATED_MARKUP = {k: v for k, v in TRANSLATED_MARKUP}

# with open(translated_markup_filename, 'wb') as handle:
#     pickle.dump(TRANSLATED_MARKUP, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(translated_markup_filename, 'rb') as handle:
    TRANSLATED_MARKUP = pickle.load(handle)

CPU times: user 27.9 ms, sys: 12 ms, total: 39.9 ms
Wall time: 39.6 ms


# Generate import payloads

In [7]:
def try_getting_key(obj, attr, subattr):
    if (hasattr(obj, attr)):
        attribute = obj.__getattribute__(attr)
        if (hasattr(attribute, subattr)):
            return attribute.__getattribute__(subattr)


def is_completed(item):
    return item.fields.status.name in ["Closed", "Resolved"]


def extract_linked_issues(linked_issue):
    if hasattr(linked_issue, "outwardIssue"):
        return {
            "key": linked_issue.outwardIssue.key,
            "relationship": linked_issue.type.outward,
            "summary": linked_issue.outwardIssue.fields.summary,
            "url": linked_issue.outwardIssue.permalink(),
            "completed": is_completed(linked_issue.outwardIssue)
        }
    else:
        return {
            "key": linked_issue.inwardIssue.key,
            "relationship": linked_issue.type.inward,
            "summary": linked_issue.inwardIssue.fields.summary,
            "url": linked_issue.inwardIssue.permalink(),
            "completed": is_completed(linked_issue.inwardIssue)
        }


def get_user_string(jira_author, jira_url):
    if jira_author.name in USER_MAPPING:
        github_id = f" / @{USER_MAPPING[jira_author.name]}"
    else:
        github_id = ""
    return f"[{jira_author.displayName}]({jira_url})" + github_id


def get_comments(issue):
    comments = []
    for comment in issue.fields.comment.comments:
        # Skip ASF GitHub Bot comments per https://github.com/apache/arrow/issues/14648
        if comment.author.name == "githubbot":
            continue

        jira_url = f"{issue.permalink()}?focusedCommentId={comment.id}"
        user_string = get_user_string(comment.author, jira_url)
        
        fixed_comments = TRANSLATED_MARKUP[issue.key]["comments"]
        comments.append({
            "body": f"{user_string}:\n{fixed_comments[comment.id]}",
            "created_at": comment.created[:-5] + "Z"
        })
    return comments


def generate_import_payload(issue):
    issue_type = ISSUETYPE_MAP[try_getting_key(issue.fields, "issuetype", "name")]
    labels = [f"Component: {c.name}" for c in issue.fields.components] + issue.fields.labels + [issue_type]
    # Filter out nonexisting labels
    labels = [label for label in labels if label in GITHUB_LABELS]
    
    # Get the earliest fix version and map it to a milestone
    fix_versions = sorted((x.name for x in issue.fields.fixVersions), key=lambda x: RELEASE_ORDER.index(x))
    fix_version = fix_versions[0] if fix_versions else None
    milestone = MILESTONE_MAP.get(fix_version, None)
    
    jira_url = issue.permalink()

    # Get watchers 
    watchers = [get_user_string(watcher, jira_url) for watcher in WATCHERS[issue.id].watchers]
    watchers = ", ".join(watchers)
   
    remote_links = [remote_link.object for remote_link in REMOTE_LINKS[issue.id]]

    body = TRANSLATED_MARKUP[issue.key]["description"] + "\n"

    if issue.fields.reporter:
        body += "\n**Reporter**: " + get_user_string(issue.fields.reporter, jira_url)
    if issue.fields.assignee:
        body += "\n**Assignee**: " + get_user_string(issue.fields.assignee, jira_url)
    if watchers:
        body += f"\n**Watchers**: {watchers}"
    
    if issue.fields.subtasks:
        body += "\n#### Subtasks:"
        for subtask in issue.fields.subtasks:
            body += f"\n- [{'X' if is_completed(subtask) else ' '}] " \
                f"[{subtask.fields.summary}]({subtask.permalink()})"

    linked_issues = [extract_linked_issues(linked_issue) for linked_issue in issue.fields.issuelinks]
    
    if linked_issues:
        body += "\n#### Related issues:"
        for li in linked_issues:
            body += \
                f"\n- #[{li['summary']}]({li['url']}) ({li['relationship']})"
    

    if issue.fields.attachment:
        body += f"\n#### Original Issue Attachments:"

        for attachment in issue.fields.attachment:
            body += f"\n- [{attachment.filename}]({attachment.content})"

    if issue.fields.customfield_12311020:
        body += "\n#### Externally tracked issue: " \
            f"[{issue.fields.customfield_12311020}]({issue.fields.customfield_12311020})"

    if remote_links:
        body += "\n#### PRs and other links:"
        for pr in remote_links:
            body += f"\n- [{pr.title}]({pr.url})"

    body += MIGRATION_NOTE.format(issue_key=issue.key, jira_url=jira_url)
    
    data = {
        "issue": {
            "title": f"{issue.fields.summary}",
            "labels": labels,
            "body": body,
            "created_at": issue.fields.created[:-5] + "Z",
            "updated_at": issue.fields.updated[:-5] + "Z",
            "closed": is_completed(issue),
        },
      "comments": get_comments(issue)
    }

    if issue.fields.resolutiondate:
        data["issue"]["closed_at"] = issue.fields.resolutiondate[:-5] + "Z"
    if milestone:
        data["issue"]["milestone"] = milestone
#     if issue.fields.assignee:
#         data["issue"]["assignee"] = USER_MAPPING.get(issue.fields.assignee.name, None)

    return data

In [8]:
all_payloads = [(issue.key, generate_import_payload(issue)) for issue in issues]

In [9]:
print(all_payloads[0][1]["issue"]["body"])

From Drill vector module

**Reporter**: [Jacques Nadeau](https://issues.apache.org/jira/browse/ARROW-1) / @jacques-n
**Assignee**: [Steven Phillips](https://issues.apache.org/jira/browse/ARROW-1) / @StevenMPhillips
**Watchers**: [Jacques Nadeau](https://issues.apache.org/jira/browse/ARROW-1) / @jacques-n

**Note**: *This issue was originally created as [ARROW-1](https://issues.apache.org/jira/browse/ARROW-1). Please see the [migration documentation](https://gist.github.com/toddfarmer/12aa88361532d21902818a6044fda4c3) for further details.*


# Import issues

In [ ]:
%%time

def import_issue(payload, session):
    while True:
        response = session.post(IMPORT_URL, json=payload, headers=GITHUB_CREDENTIALS)

        if response.status_code == 202:
            return response
        elif response.status_code == 403:
            reset_time = int(response.headers["X-RateLimit-Reset"])
            wait_time = reset_time - round(time.time() + .5)
            if wait_time > 0:
                print(f"Throttled on importing {key}, sleeping for {wait_time} seconds.")
                print("Response was: ", response.json())
                time.sleep(wait_time)
        else:
            response.raise_for_status()

def get_import_status(url, session):
    while True:
        response = session.get(url, headers=GITHUB_CREDENTIALS)

        if response.status_code == 200:
            return response
        elif response.status_code == 403:
            reset_time = int(response.headers["X-RateLimit-Reset"])
            wait_time = reset_time - round(time.time() + .5)
            if wait_time > 0:
                print(f"Throttled on getting {url}, sleeping for {wait_time} seconds.")
                print("Response was: ", response.json())
                time.sleep(wait_time)
        else:
            response.raise_for_status()

payloads = all_payloads
import_responses = {}

with requests.Session() as s:
    for i, (key, payload) in enumerate(payloads):
        if (i % 100 == 0):
            print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] importing ", i, "/", len(payloads))
        import_responses[key] = {"import_response": import_issue(payload, s), "status": ""}

In [ ]:
%%time

# Check for import statuses 
with requests.Session() as s:
    for key in import_responses.keys():
        status_url = import_responses[key]["import_response"].json()["url"]

        if import_responses[key]["status"] != "imported":
            response = get_import_status(status_url, s)
            import_responses[key]["status"] = response.json()["status"]

            if import_responses[key]["status"] == "imported":
                import_responses[key]["issue_url"] = \
                    response.json()["issue_url"].replace("https://api.github.com/repos/", "https://github.com/")

In [ ]:
from IPython.display import display, HTML

for url in [x['issue_url'] for x in import_responses.values() if 'issue_url' in x]:
    display(HTML("""<a href="{}">{}</a>""".format(url, url)))

# todo
* subscribe people?
* Issue type

# Process

1. Create issue with comments included
2. Collect key to github url map
3. Post jira comments with github links
4. Lock Jira
5. Update github issue with corrected related issue / subtask links